# Pydantic generate models

Test of generating pydantic data models from openapi spec and posting NGSI-LD broker. First generate the models:

In [16]:
!datamodel-codegen --input models.yaml --output models.py

In [17]:
import datetime
import requests
from ngsildclient import Entity
from models import AnimalObservation, HarvestEvent

In [18]:
start = datetime.datetime.fromisoformat("2023-01-01T00:00:00")
a1 = AnimalObservation(methaneyield=20, net_energy_requirement=500, 
                  observedAt = start.isoformat() + "Z", observationDate = start.isoformat() + "Z" )
e = a1.dict(exclude_none=True)
e["type"] = "AnimalObservation"
e["id"] = Entity._build_fully_qualified_id(e["type"], "cow006")

In [20]:
et = Entity.from_dict(e)
et.pprint()

{
  "methaneyield": 20.0,
  "net_energy_requirement": 500.0,
  "observedAt": "2023-01-01T00:00:00Z",
  "observationDate": "2023-01-01T00:00:00Z",
  "type": "AnimalObservation",
  "id": "urn:ngsi-ld:AnimalObservation:cow006",
  "@context": [
    "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"
  ]
}

In [6]:
import requests
ctx = "https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context-v1.6.jsonld"
headers = {'Link': f'<{ctx}>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"'}
r = requests.post("http://localhost:1026/ngsi-ld/v1/entities", 
                  headers=headers,
                  json=e)
print(r.status_code)
print(r.text)

201



In [7]:
r = requests.get(f"http://localhost:1026/ngsi-ld/v1/entities/{e['id']}", 
                 params={"options" : "keyValues"})
AnimalObservation(**r.json())
r.json()

{'id': 'urn:ngsi-ld:AnimalObservation:cow006',
 'type': 'AnimalObservation',
 'methaneyield': 20,
 'net_energy_requirement': 500,
 'observedAt': '2023-01-01T00:00:00Z',
 'observationDate': '2023-01-01T00:00:00Z'}

# Temporal

In [8]:
start = datetime.datetime.fromisoformat("2023-01-01T00:00:00")
e = a1.dict(exclude_none=True)
e["type"] = "AnimalObservation"
e["id"] = Entity._build_fully_qualified_id(e["type"], "cow006")
for t in range(3):
    e["observedAt"] = (start + datetime.timedelta(days=t)).isoformat() + "Z"
    r = requests.post(f"http://localhost:1026/ngsi-ld/v1/entityOperations/update", 
                  headers=headers,
                  json=[e])
    print(r.status_code)
    #print(r.text)


204
204
204


In [12]:
url = 'http://localhost:8080/temporal/entities/' 
headers = {
    'Link': f'<{ctx}>; rel="http://www.w3.org/ns/json-ld#context"; type="application/ld+json"',
}
params = {
    'type': e["type"],
    'pageSize': 100,
    'lastN': 10,
    'timeproperty': 'observedAt', # TODO add observedAt to properties <- code in 
    #'timeproperty': 'createdAt',
    'timerel': 'before',
    'timeAt': f'{datetime.datetime.now().isoformat()}',
    'options' : 'temporalValues'
}

response = requests.get(url, headers=headers, params=params)
response.json()

[]

In [25]:
import yaml
print(yaml.dump(AnimalObservation.schema()))

properties:
  methaneyield:
    description: Methane (x / d)
    title: Methaneyield
    type: number
  net_energy_requirement:
    description: http://opendata.inrae.fr/ATOL/page/ATOL_0001601
    title: Net Energy Requirement
    type: number
  observationDate:
    description: Date of the observation
    title: Observationdate
    type: string
  observedAt:
    description: Date of the observation
    title: Observedat
    type: string
title: AnimalObservation
type: object

